<a href="https://colab.research.google.com/github/MushroomFleet/Reverse-Coco/blob/main/ReverseCocoV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![ko-fi](https://img.shields.io/badge/Support%20me%20on%20Ko--fi-F16061?logo=ko-fi&logoColor=white&style=flat)](https://ko-fi.com/driftjohnson)

# **Johnsons Tools / Colab Notebooks For Augmented Productivity**

| Notebook Name | Description | Link |
| --- | --- | --- | 
| [Caption Append](https://github.com/MushroomFleet/Caption-Append) | Caption Append | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://colab.research.google.com/github/MushroomFleet/Caption-Append/blob/main/Caption_Append_V1.ipynb) |
| [Caption Concat](https://github.com/MushroomFleet/Caption-Concat) | Caption Concat | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://colab.research.google.com/github/MushroomFleet/Caption-Concat/blob/main/Caption_Concat_V1_2.ipynb) |
| [Caption Stripper](https://github.com/MushroomFleet/Caption-Stripper) | Caption Stripper | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://colab.research.google.com/github/MushroomFleet/Caption-Stripper/blob/main/Caption_Stripper_V1.ipynb) |
| [Caption Swap](https://github.com/MushroomFleet/Caption-Swap) | Caption Swap | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://colab.research.google.com/github/MushroomFleet/Caption-Swap/blob/main/Caption_Swap_V1.ipynb) | 
| [Reverse Coco](https://github.com/MushroomFleet/Reverse-Coco) | Reverse Coco | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://colab.research.google.com/github/MushroomFleet/Reverse-Coco/blob/main/ReverseCocoV1.ipynb) | 
| [FFXFade-o-rama](https://github.com/MushroomFleet/FFXFADE-O-RAMA) | FFX Fade o rama | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://colab.research.google.com/github/MushroomFleet/FFXFADE-O-RAMA/blob/main/djz_FFXFADE_O_RAMA.ipynb) | 
| [Deforum Tools](https://github.com/MushroomFleet/Deforum-Sequence-Tools) | Deforum Tools | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://colab.research.google.com/github/MushroomFleet/Deforum-Sequence-Tools/blob/main/Deforum_Sequence_ToolsV1.ipynb) | 

###**Google Drive Sync & setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


using python we can interrogate Coco Model to draw out image descriptions if a match is found. We can select a list of words and choose the min/max length params. Also you can narrow the search with ensuring the number of matched words. If you cannot find any matches, use common listed words, add more listed words or reduce "Num_Words". 

If you use Wildcards then enable the Underscore mode to surround your listed words with double underscores.

# Required


In [ ]:
import random
import json
import os
import re

import zipfile
import spacy

nlp = spacy.load("en_core_web_sm")

def install(package):
    # Install a package using pip
    import subprocess
    subprocess.check_call(["!pip", "install", package])

# Install wget and unzip if they are not already installed
try:
    import wget
except ImportError:
    !pip install wget
try:
    import zipfile
except ImportError:
    !pip install unzip

def download_dataset():
    if not os.path.exists("annotations"):
        os.makedirs("annotations")
    wget.download("http://images.cocodataset.org/annotations/annotations_trainval2014.zip", out="annotations/")
    with zipfile.ZipFile("annotations/annotations_trainval2014.zip", "r") as zip_ref:
        zip_ref.extractall("annotations/")

# Config

In [ ]:
# Define variables
words = "danger,cat,attack" # @param {type:'string'}
words_min = 4 # @param {type:'integer'}
words_max = 20 # @param {type:'integer'}
num_words = 2 # @param {type:'integer'}

underscore = True # @param {type:'boolean'}

file_path = "/content/descriptions.txt" # @param {type:'string'}



def generate_description(words, words_min, words_max, num_words, underscore=False):
    if not os.path.exists("annotations/annotations/captions_train2014.json"):
        download_dataset()
    with open("annotations/annotations/captions_train2014.json", "r") as f:
        data = json.load(f)
    descriptions = [d["caption"] for d in data["annotations"]]
    matching_descriptions = []
    for description in descriptions:
        count = 0
        for word in words:
            if word in description:
                count += 1
                if count >= num_words:
                    matching_descriptions.append(description)
                    break
    if not matching_descriptions:
        return None
    description = random.choice(matching_descriptions)
    description = description.strip().capitalize() + "."
    
    # Use spaCy to improve grammar and syntax
    doc = nlp(description)
    description = " ".join([token.text for token in doc])
    
    if underscore:
        # Add double underscores to the listed words
        for word in words:
            description = re.sub(r'\b%s\b' % word, "__%s__" % word, description)

    # Limit the number of words in the description
    words_in_description = description.split()
    if len(words_in_description) < words_min:
        return None
    if len(words_in_description) > words_max:
        description = " ".join(words_in_description[:words_max]) + "."
        
    # Remove any text containing two periods that are touching
    description = re.sub(r'\.\.+', '.', description)
    
    return description

description = None
while not description:
    description = generate_description(words.split(","), words_min, words_max, num_words, underscore)

print(description)

A black __cat__ is attacking a yellow umbrella .


# Write Result to Text File

In [ ]:
print(description)

if os.path.exists(file_path):
    with open(file_path, "a") as f:
        f.write("\n" + description)
else:
    with open(file_path, "w") as f:
        f.write(description)

A black __cat__ is attacking a yellow umbrella .


# if not working google drive:
Remember to download the description.txt ! it is lost when the session ends.